### Instalando pacotes

In [1]:
!pip install openpyxl
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import openpyxl

# Avaliação 03
Grupo 5:
*   Allane
*   Item da lista



### Instanciando os grafos
Estamos usando DataFrames da biblioteca pandas para representar as matrizes de adjacencia dos grafos.

In [3]:
matriz_problema_km = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Km")
matriz_problema_min = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Min")
cidades = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Cidades")


### Estrutura da Busca Local

In [4]:
# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        custo += grafo[v1][v2 - 1]
    return custo

# O argumento 'heuristica' eh o algoritmo da busca local que sera realizada (shift, swap etc).
# ele deve retornar uma vizinhanca - a lista de solucoes geradas
def busca_local_primeira_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = custo_original
    solucao_atual = solucao
    melhoria = True
    while melhoria:
        melhoria = False
        for nova_solucao in heuristica(solucao_atual):
            novo_custo = calcular_custo(nova_solucao, grafo)
            if novo_custo < custo_atual:
                print("Novo custo: ", novo_custo)
                solucao_atual = nova_solucao
                custo_atual = novo_custo
                melhoria = True
                break
    return solucao_atual, custo_atual


### Heurísticas

In [5]:
def shift(solucao : list):
    solucao_cp = solucao[:]
    for idx_vert_shift in range(1, len(solucao) - 1):
        for idx_destino_shift in range(2, len(solucao) - 1):
            vert_shift = solucao_cp.pop(idx_vert_shift)
            solucao_cp.insert(idx_destino_shift, vert_shift)
            yield solucao_cp

def swap(solucao : list):
    solucao_cp = solucao[:]
    for idx_vert_swap in range(1, len(solucao) - 1):
        for idx_destino_swap in range(2, len(solucao) - 1):
            vert_swap = solucao_cp[idx_vert_swap]
            solucao_cp[idx_vert_swap] = solucao_cp[idx_destino_swap]
            solucao_cp[idx_destino_swap] = vert_swap
            # print("Solucao do swap : ", solucao_cp)
            yield solucao_cp

def inversao(solucao):
    solucao_cp = solucao[:]
    for idx_vert_inicio in range(1, len(solucao) - 1):
        for idx_destino in range(2, len(solucao) - 1):
            sublista_invertida = solucao_cp[idx_vert_inicio:idx_destino]
            sublista_invertida.reverse()
            yield solucao_cp[0:idx_vert_inicio] + sublista_invertida + solucao_cp[idx_destino:]


## Busca por vizinho mais próximo

In [18]:
# Estamos considerando o label numerico
def vizinho_mais_proximo(grafo : pd.DataFrame, vertice_inicial : int | str):
    # Criando conjunto de vertices nao visitados
    # o conjunto contem os indices referentes ao vertice na matriz problema
    vertices_nao_visitados = set(grafo.index)
    percurso = [vertice_inicial]
    vertices_nao_visitados.remove(vertice_inicial - 1)
    dist_total = 0
    v_atual = vertice_inicial
    while len(vertices_nao_visitados) > 0:
        # Obtem o label (numero) do vertice com menor custo
        idx_vizinho_mais_proximo = grafo[v_atual].loc[list(vertices_nao_visitados)].idxmin(skipna=True)
        prox = idx_vizinho_mais_proximo + 1
        dist_total += grafo[v_atual][idx_vizinho_mais_proximo]
        percurso.append(int(prox))
        v_atual = prox
        vertices_nao_visitados.remove(idx_vizinho_mais_proximo)
    dist_total += grafo[v_atual][vertice_inicial -1]
    percurso.append(vertice_inicial)
    return percurso, dist_total


## Aplicando nos problemas

In [ ]:
def aplicar_problema(grafo, inicial, heuristica_construtiva, heuristicas_busca_local):
    print("Aplicando Heuristica Construtiva: ", heuristica_construtiva.__name__)
    caminho, custo = heuristica_construtiva(grafo, inicial)
    print(f"Caminho: {caminho}")
    print(f"Custo: {custo}")

    print("Aplicando busca(s) local(is):")

    print("Aplicando busca(s) local(is):")
    for h in heuristicas_busca_local:
        print(f"Heuristica : {h.__name__}")
        print("Tipo : Primeira Melhoria")
        solucao_local, custo_local = busca_local_primeira_melhoria(caminho, custo, h, grafo)
        print(f"Caminho pos busca local: {solucao_local}")
        print(f"Custo pos busca local: {custo_local}")

    print("Aplicando busca local:")
    print("Heuristica : shift")
    print("Tipo : Primeira Melhoria")
    solucao_shift, custo_shift = busca_local_primeira_melhoria(caminho, custo, shift, matriz_problema_km)
    print(f"Caminho pos busca local: {solucao_shift}")
    print(f"Custo pos busca local: {custo_shift}")

    print("Aplicando busca local:")
    print("Heuristica : inversao")
    print("Tipo : Primeira Melhoria")
    solucao_inversao, custo_inversao = busca_local_primeira_melhoria(caminho, custo, inversao, matriz_problema_km)
    print(f"Caminho pos busca local: {solucao_inversao}")
    print(f"Custo pos busca local: {custo_inversao}")


### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [7]:
print("Testando a busca por vizinho mais proximo")
caminho, custo = vizinho_mais_proximo(matriz_problema_km, 1)
print(f"Caminho: {caminho}")
print(f"Custo: {custo}")

print("Aplicando busca local:")

print("Heuristica : swap")
print("Tipo : Primeira Melhoria")
solucao_swap, custo_swap = busca_local_primeira_melhoria(caminho, custo, swap, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_swap}")
print(f"Custo pos busca local: {custo_swap}")

print("Aplicando busca local:")
print("Heuristica : shift")
print("Tipo : Primeira Melhoria")
solucao_shift, custo_shift = busca_local_primeira_melhoria(caminho, custo, shift, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_shift}")
print(f"Custo pos busca local: {custo_shift}")

print("Aplicando busca local:")
print("Heuristica : inversao")
print("Tipo : Primeira Melhoria")
solucao_inversao, custo_inversao = busca_local_primeira_melhoria(caminho, custo, inversao, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_inversao}")
print(f"Custo pos busca local: {custo_inversao}")


Testando a busca por vizinho mais proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo: 2291.3999999999996
Aplicando busca local:
Heuristica : swap
Tipo : Primeira Melhoria
Caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo pos busca local: 2291.3999999999996
Aplicando busca local:
Heuristica : shift
Tipo : Primeira Melhoria
Caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo pos busca local: 2291.3999999999996
Aplicando busca local:
Heuristica : inversao
Tipo : Primeira Melhoria
Novo custo:  2282.7
Novo 

### Problema 2
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pelo tempo em minutos.


In [8]:
print("Testando a busca por vizinho mais proximo")
caminho, custo = vizinho_mais_proximo(matriz_problema_min, 1)
print(f"Caminho: {caminho}")
print(f"Custo: {custo}")

print("Aplicando busca local:")
print("Heuristica : swap")
print("Tipo : Primeira Melhoria")
solucao_swap, custo_swap = busca_local_primeira_melhoria(caminho, custo, swap, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_swap}")
print(f"Custo pos busca local: {custo_swap}")

print("Aplicando busca local:")
print("Heuristica : shift")
print("Tipo : Primeira Melhoria")
solucao_shift, custo_shift = busca_local_primeira_melhoria(solucao_swap, custo_swap, shift, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_shift}")
print(f"Custo pos busca local: {custo_shift}")


print("Aplicando busca local:")
print("Heuristica : inversao")
print("Tipo : Primeira Melhoria")
solucao_inversao, custo_inversao = busca_local_primeira_melhoria(solucao_shift, custo_shift, inversao, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_inversao}")
print(f"Custo pos busca local: {custo_inversao}")



Testando a busca por vizinho mais proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo: 2355.0
Aplicando busca local:
Heuristica : swap
Tipo : Primeira Melhoria
Caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo pos busca local: 2355.0
Aplicando busca local:
Heuristica : shift
Tipo : Primeira Melhoria
Caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo pos busca local: 2355.0
Aplicando busca local:
Heuristica : inversao
Tipo : Primeira Melhoria
Caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23

### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância
em km

In [20]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

print(matriz_problema_km_36)

caminho, custo = vizinho_mais_proximo(matriz_problema_km_36, 1)
print(f"Caminho: {caminho}")
print(f"Custo: {custo}")

print("Aplicando busca local:")

print("Heuristica : swap")
print("Tipo : Primeira Melhoria")
solucao_swap, custo_swap = busca_local_primeira_melhoria(caminho, custo, swap, matriz_problema_km_36)
print(f"Caminho pos busca local: {solucao_swap}")
print(f"Custo pos busca local: {custo_swap}")

print("Aplicando busca local:")
print("Heuristica : shift")
print("Tipo : Primeira Melhoria")
solucao_shift, custo_shift = busca_local_primeira_melhoria(caminho, custo, shift, matriz_problema_km_36)
print(f"Caminho pos busca local: {solucao_shift}")
print(f"Custo pos busca local: {custo_shift}")

print("Aplicando busca local:")
print("Heuristica : inversao")
print("Tipo : Primeira Melhoria")
solucao_inversao, custo_inversao = busca_local_primeira_melhoria(caminho, custo, inversao, matriz_problema_km_36)
print(f"Caminho pos busca local: {solucao_inversao}")
print(f"Custo pos busca local: {custo_inversao}")

       1      2      3      4      5      6      7      8      9      10  ...  \
0     NaN   38.8   29.7   37.4   76.8  106.0  170.0   26.4   42.4   26.8  ...   
1    38.8    NaN   31.5   39.3   50.7  108.0  205.0   60.9   77.0   62.4  ...   
2    29.7   31.5    NaN    9.9   82.2   79.0  196.0   51.8   67.8   53.2  ...   
3    37.4   39.3    9.9    NaN   73.3   73.4  204.0   59.5   75.6   60.9  ...   
4    76.8   50.7   82.2   73.3    NaN  228.0  128.0   99.3  115.0   83.6  ...   
5   106.0  108.0   79.0   73.4  228.0    NaN  272.0  128.0  144.0  130.0  ...   
6   170.0  205.0  196.0  204.0  128.0  272.0    NaN  166.0  151.0  159.0  ...   
7    26.4   60.9   51.8   59.5   99.3  128.0  166.0    NaN   16.1   28.1  ...   
8    42.4   77.0   67.8   75.6  115.0  144.0  151.0   16.1    NaN   27.3  ...   
9    26.8   62.4   53.2   60.9   83.6  130.0  159.0   28.1   27.3    NaN  ...   
10   49.9   43.2   67.7   82.3   51.1  151.0  184.0   71.2   62.3   35.1  ...   
11   45.9   80.5   71.3   79

### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto
em minutos.


In [21]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

caminho, custo = vizinho_mais_proximo(matriz_problema_min_36, 1)
print(f"Caminho: {caminho}")
print(f"Custo: {custo}")

print("Aplicando busca local:")

print("Heuristica : swap")
print("Tipo : Primeira Melhoria")
solucao_swap, custo_swap = busca_local_primeira_melhoria(caminho, custo, swap, matriz_problema_min_36)
print(f"Caminho pos busca local: {solucao_swap}")
print(f"Custo pos busca local: {custo_swap}")

print("Aplicando busca local:")
print("Heuristica : shift")
print("Tipo : Primeira Melhoria")
solucao_shift, custo_shift = busca_local_primeira_melhoria(caminho, custo, shift, matriz_problema_min_36)
print(f"Caminho pos busca local: {solucao_shift}")
print(f"Custo pos busca local: {custo_shift}")

print("Aplicando busca local:")
print("Heuristica : inversao")
print("Tipo : Primeira Melhoria")
solucao_inversao, custo_inversao = busca_local_primeira_melhoria(caminho, custo, inversao, matriz_problema_min_36)
print(f"Caminho pos busca local: {solucao_inversao}")
print(f"Custo pos busca local: {custo_inversao}")


       1      2      3      4      5      6      7      8      9      10  ...  \
0     NaN   38.8   29.7   37.4   76.8  106.0  170.0   26.4   42.4   26.8  ...   
1    38.8    NaN   31.5   39.3   50.7  108.0  205.0   60.9   77.0   62.4  ...   
2    29.7   31.5    NaN    9.9   82.2   79.0  196.0   51.8   67.8   53.2  ...   
3    37.4   39.3    9.9    NaN   73.3   73.4  204.0   59.5   75.6   60.9  ...   
4    76.8   50.7   82.2   73.3    NaN  228.0  128.0   99.3  115.0   83.6  ...   
5   106.0  108.0   79.0   73.4  228.0    NaN  272.0  128.0  144.0  130.0  ...   
6   170.0  205.0  196.0  204.0  128.0  272.0    NaN  166.0  151.0  159.0  ...   
7    26.4   60.9   51.8   59.5   99.3  128.0  166.0    NaN   16.1   28.1  ...   
8    42.4   77.0   67.8   75.6  115.0  144.0  151.0   16.1    NaN   27.3  ...   
9    26.8   62.4   53.2   60.9   83.6  130.0  159.0   28.1   27.3    NaN  ...   
10   49.9   43.2   67.7   82.3   51.1  151.0  184.0   71.2   62.3   35.1  ...   
11   45.9   80.5   71.3   79

### Insercao Mais Barata

In [134]:
import math

# Estamos considerando o label numerico
def insercao_mais_barata(grafo : pd.DataFrame, vertice_inicial : int | str):

    vertices_restantes = set(grafo.index)

    # Selecionamos o indice do vertice mais proximo a raiz de forma gulosa
    v_mais_proximo = grafo[vertice_inicial].idxmin()

    # Ciclo inicial
    rota = [vertice_inicial, v_mais_proximo + 1, vertice_inicial]
    vertices_restantes.remove( vertice_inicial - 1, )
    vertices_restantes.remove(v_mais_proximo)
    vertices_inseridos = {vertice_inicial - 1, v_mais_proximo}

    custo_total = 0
    while vertices_restantes:
        # Estamos selecionando o vertice de 'vertices_restantes' que tem a menor distancia a outro vertice
        # Filtra apenas as linhas da matriz referentes aos vertices do conjunto de vertices_restantes
        r = grafo[list(vertices_restantes)].iloc[list(vertices_inseridos)].min().idxmin()

        # Consegue o indice do vertice do conjunto 'vertices_restantes' que tem a menor distancia
        # ao algum vertice incluido na rota. Para manter este codigo similar ao pseudocodigo visto em sala, chamaremos a variavel de
        # Encontrar a melhor posicao para inserir a rota
        melhor_custo_insercao = math.inf
        melhor_posicao_insercao = None
        for i in range(1, len(rota) - 1):
            u, v = rota[i] - 1, rota[i + 1] - 1
            # print(f"u : {u}, v : {v}")
            # print("Custo na matriz : ", grafo[u + 1][v])
            custo = grafo[u + 1][r] + grafo[r + 1][v] - grafo[u + 1][v]
            if custo < melhor_custo_insercao:
                melhor_custo_insercao = custo
                melhor_posicao_insercao = i

        rota.insert(melhor_posicao_insercao + 1, r + 1)
        vertices_restantes.remove(r)
        vertices_inseridos.add(r)
        custo_total += melhor_custo_insercao
    return rota, calcular_custo(rota, grafo)


In [135]:
print("Testando a busca por vizinho mais proximo")
caminho, custo = insercao_mais_barata(matriz_problema_km, 1)
print(f"Caminho: {caminho}")
print(f"Custo: {custo}")

print("Aplicando busca local:")
print("Heuristica : swap")
print("Tipo : Primeira Melhoria")
solucao_swap, custo_swap = busca_local_primeira_melhoria(caminho, custo, swap, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_swap}")
print(f"Custo pos busca local: {custo_swap}")

print("Aplicando busca local:")
print("Heuristica : shift")
print("Tipo : Primeira Melhoria")
solucao_shift, custo_shift = busca_local_primeira_melhoria(solucao_swap, custo_swap, shift, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_shift}")
print(f"Custo pos busca local: {custo_shift}")


print("Aplicando busca local:")
print("Heuristica : inversao")
print("Tipo : Primeira Melhoria")
solucao_inversao, custo_inversao = busca_local_primeira_melhoria(solucao_shift, custo_shift, inversao, matriz_problema_km)
print(f"Caminho pos busca local: {solucao_inversao}")
print(f"Custo pos busca local: {custo_inversao}")


Testando a busca por vizinho mais proximo
Caminho: [1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo: 2148.1
Aplicando busca local:
Heuristica : swap
Tipo : Primeira Melhoria
Novo custo:  2148.0
Caminho pos busca local: [1, 23, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Custo pos busca local: 2148.0
Aplicando busca local:
Heuristica : shift
Tipo : Primeira Melhoria
Novo custo:  2147.2
Caminho pos busca local: [1, 23, 21, 8, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Custo pos busca local: 2147.2
Aplicando busca local:
Heuristica : inversao
Tipo : Primeira Melhoria
Novo custo:  2143.3
N